# Running dataset with basic convolutional model

If you want to see some basics about the data and the images, refer to the other notebooks

In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torchvision.transforms as transforms
from torch.utils.data import random_split
import torch.optim as optim

In [2]:
%load_ext autoreload
%autoreload 2

import data_handling as data
import model

## Creating dataset

The code for this dataset is included in ```data_handling.py```. 

In [3]:
needle_directory = '/Users/carlosolivares/be224-fp/data/NeedleImages/'
labels_path = os.path.join(needle_directory, 'Labels.csv')

data_transformer = transforms.Compose([transforms.ToTensor()])

dataset = data.NeedleImageDataset(
    path2data=needle_directory,
    path2labels=labels_path,
    transform = data_transformer
)

In [4]:
len_dataset = len(dataset)
len_train = int(0.8 * len_dataset)
len_val = len_dataset - len_train

train_ds, val_ds = random_split(dataset, [len_train, len_val])

print("train datset length:", len(train_ds))
print("validation dataset length:", len(val_ds))

train_dl = DataLoader(train_ds, batch_size=10, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=10, shuffle=False)

train datset length: 504
validation dataset length: 127


In [5]:
print(len(train_dl))

51


In [6]:
"""
checking to see that the batches come appropriately shaped for input to model. 
The model accepts data in (batch, channels, height, width)
Our images are grayscale, so we stack the values to get 3 channels like an RGB
"""

for xb, yb in val_dl:
    print(xb.shape)
    print(yb.shape)
    break

torch.Size([10, 3, 512, 512])
torch.Size([10])


## Getting model working

ummm

In [7]:
naivenet = model.NaiveNet()

In [8]:
for xb, yb in val_dl:
    out = naivenet(xb)
    break

In [9]:
out.shape

torch.Size([10, 2])

## Training the model for two epoch to see if it actually learns



In [10]:
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
optimizer = optim.SGD(naivenet.parameters(), lr=0.001, momentum=0.9)

In [11]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        naivenet.train()
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = naivenet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0
    val_loss = model.get_validation_loss(naivenet, criterion, val_dl)
    print('validation loss after epoch {}: {}'.format((i+1), val_loss))

print('Finished Training')

[1,    10] loss: 6.717
[1,    20] loss: 6.236
[1,    30] loss: 6.087
[1,    40] loss: 6.721
[1,    50] loss: 6.269
validation loss after epoch 51: 0.6045545048601045
[2,    10] loss: 6.422
[2,    20] loss: 5.959
[2,    30] loss: 6.286
[2,    40] loss: 5.813
[2,    50] loss: 6.450
validation loss after epoch 51: 0.6030995714382863
[3,    10] loss: 6.042
[3,    20] loss: 5.014
[3,    30] loss: 6.449
[3,    40] loss: 6.890
[3,    50] loss: 6.517
validation loss after epoch 51: 0.6068611501708744
[4,    10] loss: 5.790
[4,    20] loss: 6.106
[4,    30] loss: 5.782
[4,    40] loss: 6.559
[4,    50] loss: 6.631
validation loss after epoch 51: 0.6050305629339744
[5,    10] loss: 5.819
[5,    20] loss: 6.350
[5,    30] loss: 6.349
[5,    40] loss: 6.502
[5,    50] loss: 5.756
validation loss after epoch 51: 0.6048059576139675
[6,    10] loss: 6.352
[6,    20] loss: 6.135
[6,    30] loss: 6.121
[6,    40] loss: 6.124
[6,    50] loss: 6.190
validation loss after epoch 51: 0.6040606968046174
[7, 

In [12]:
## Ok, it seems to be working, so now all we need is to figure out exactly how to get metrics and loss or whatever

In [13]:
## investigating how the crossentrupy loss works. From the docs: 

# Example of target with class indices
loss = torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
#output.backward()
# Example of target with class probabilities
#input = torch.randn(3, 5, requires_grad=True)
#target = torch.randn(3, 5).softmax(dim=1)
#output = loss(input, target)
#output.backward()

In [14]:
input

tensor([[-0.6759, -0.8502,  0.7985, -0.5464,  1.2984],
        [-1.1789, -1.6076, -0.6740,  1.3447, -0.6631],
        [-1.3812,  0.5992,  2.3664,  0.8805,  0.3675]], requires_grad=True)

In [15]:
target

tensor([1, 0, 2])

In [16]:
output

tensor(2.0512, grad_fn=<NllLossBackward>)

In [17]:
outputs

tensor([[-0.4508,  0.3201],
        [-0.4884,  0.3111],
        [-0.4619,  0.3550],
        [-0.5006,  0.3499]], grad_fn=<AddmmBackward>)

In [18]:
labels

tensor([0, 0, 1, 1])

In [19]:
loss(outputs, labels)

tensor(0.7609, grad_fn=<NllLossBackward>)

In [20]:
len(val_dl.dataset)

127

In [ ]:
# the next thing to do is actually get the metrics from model and stuff. 
# looks like you might need an lr schedulaer